In [1]:
!nvidia-smi

Tue Aug 26 13:29:41 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   55C    P8             13W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
%%shell
# Adapted from the official dockerfile: https://github.com/google-deepmind/alphafold3/blob/main/docker/Dockerfile
rm -rf /content/sample_data

# /app/alphafold - alphafold3 git repo checkout
mkdir /app
cd /app
git clone https://github.com/google-deepmind/alphafold3.git alphafold
apt-get install --yes --quiet python3.12 python3-pip python3.12-venv python3.12-dev git wget gcc g++ make zlib1g-dev zstd

# /alphafold3_venv - pip environment for alphafold3 dependancies
python3 -m venv /alphafold3_venv
source /alphafold3_venv/bin/activate

# /hmmer - build hmmer from source & custom patch
mkdir /hmmer_build /hmmer
wget http://eddylab.org/software/hmmer/hmmer-3.4.tar.gz --directory-prefix /hmmer_build
cd /hmmer_build && echo "ca70d94fd0cf271bd7063423aabb116d42de533117343a9b27a65c17ff06fbf3 hmmer-3.4.tar.gz" | sha256sum --check
cd /hmmer_build && tar zxf hmmer-3.4.tar.gz && rm hmmer-3.4.tar.gz

cp /app/alphafold/docker/jackhmmer_seq_limit.patch /hmmer_build/
cd /hmmer_build && patch -p0 < jackhmmer_seq_limit.patch

cd /hmmer_build/hmmer-3.4 && ./configure --prefix /hmmer
cd /hmmer_build/hmmer-3.4 && make -j
cd /hmmer_build/hmmer-3.4 && make install
cd /hmmer_build/hmmer-3.4/easel && make install
rm -R /hmmer_build

# build pip dependencies & ligand dictionary
cd /app/alphafold
pip3 install --no-cache-dir setuptools
pip3 install --no-cache-dir -r dev-requirements.txt
pip3 install --no-cache-dir --no-deps .
build_data

Cloning into 'alphafold'...
remote: Enumerating objects: 1443, done.
remote: Counting objects: 100% (904/904), done.
remote: Compressing objects: 100% (402/402), done.
remote: Total 1443 (delta 654), reused 502 (delta 502), pack-reused 539 (from 3)
Receiving objects: 100% (1443/1443), 39.42 MiB | 23.87 MiB/s, done.
Resolving deltas: 100% (850/850), done.
Reading package lists...
Building dependency tree...
Reading state information...
g++ is already the newest version (4:11.2.0-1ubuntu1).
g++ set to manually installed.
gcc is already the newest version (4:11.2.0-1ubuntu1).
gcc set to manually installed.
make is already the newest version (4.3-4.1build1).
make set to manually installed.
git is already the newest version (1:2.34.1-1ubuntu1.15).
wget is already the newest version (1.21.2-2ubuntu1.1).
zlib1g-dev is already the newest version (1:1.2.11.dfsg-2ubuntu9.2).
zlib1g-dev set to manually installed.
python3.12 is already the newest version (3.12.11-1+jammy1).
python3.12-dev is alrea

In [3]:
# Copy weights from Google Drive to the default location at /root/models/af3.bin.zst
from google.colab import drive
drive.mount('/content/drive')
!mkdir -p /root/models/
!cp /content/drive/MyDrive/alphafold3/models/af3.bin.zst /root/models/af3.bin.zst
!md5sum /root/models/af3.bin.zst

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
ea076e2d05168d66a3d9a3fa62e029d0  /root/models/af3.bin.zst


In [4]:
%%shell
python3 -m venv /colabfold_venv
source /colabfold_venv/bin/activate
pip3 install "colabfold[alphafold] @ git+https://github.com/sokrypton/ColabFold.git"

  Cloning https://github.com/sokrypton/ColabFold.git to /tmp/pip-install-16xsem0q/colabfold_aaa23cfdf753434bb83fe5415db202a7
  Running command git clone --filter=blob:none --quiet https://github.com/sokrypton/ColabFold.git /tmp/pip-install-16xsem0q/colabfold_aaa23cfdf753434bb83fe5415db202a7
  Resolved https://github.com/sokrypton/ColabFold.git to commit dfbe59da74e9c730a0af97c78a6e11dd336153ac
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached numpy-2.3.2-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 95.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 127.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 186.1 MB/s eta 0:00:00
Using cached numpy-2.3.2-cp312-cp312-manylinux_2_27_x86_64.manylin

In [33]:
%%shell
mkdir -p /content/alphafold3_examples/fasta
cd /content/alphafold3_examples/fasta
wget https://rest.uniprot.org/uniprotkb/O00244.fasta
wget https://rest.uniprot.org/uniprotkb/P0CG48.fasta
wget https://rest.uniprot.org/uniprotkb/Q01973.fasta

--2025-08-26 14:12:17--  https://rest.uniprot.org/uniprotkb/O00244.fasta
Resolving rest.uniprot.org (rest.uniprot.org)... 193.62.193.81
Connecting to rest.uniprot.org (rest.uniprot.org)|193.62.193.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 167 [text/plain]
Saving to: ‘O00244.fasta’

O00244.fasta        100%[===================>]     167  --.-KB/s    in 0s      

2025-08-26 14:12:18 (124 MB/s) - ‘O00244.fasta’ saved [167/167]

--2025-08-26 14:12:18--  https://rest.uniprot.org/uniprotkb/P0CG48.fasta
Resolving rest.uniprot.org (rest.uniprot.org)... 193.62.193.81
Connecting to rest.uniprot.org (rest.uniprot.org)|193.62.193.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 775 [text/plain]
Saving to: ‘P0CG48.fasta’

P0CG48.fasta        100%[===================>]     775  --.-KB/s    in 0s      

2025-08-26 14:12:19 (587 MB/s) - ‘P0CG48.fasta’ saved [775/775]

--2025-08-26 14:12:19--  https://rest.uniprot.org/uniprotkb/Q01973.fasta

In [34]:
%%shell
MPLBACKEND=AGG
source /colabfold_venv/bin/activate
mkdir -p /content/alphafold3_examples/colabfold_msas
colabfold_batch --msa-only --af3-json /content/alphafold3_examples/fasta/O00244.fasta /content/alphafold3_examples/colabfold_msas
colabfold_batch --msa-only --af3-json /content/alphafold3_examples/fasta/P0CG48.fasta /content/alphafold3_examples/colabfold_msas
colabfold_batch --msa-only --af3-json /content/alphafold3_examples/fasta/Q01973.fasta /content/alphafold3_examples/colabfold_msas

2025-08-26 14:12:42,451 Running colabfold 1.5.5 (dfbe59da74e9c730a0af97c78a6e11dd336153ac)

limited shared resource only capable of processing a few thousand MSAs per day. Please
submit jobs only from a single IP address. We reserve the right to limit access to the
server case-by-case when usage exceeds fair use. If you require more MSAs: You can 
precompute all MSAs with `colabfold_search` or host your own API and pass it to `--host-url`

COMPLETE: 100% 150/150 [00:01<00:00, 114.22it/s]
2025-08-26 14:12:45,341 Running colabfold 1.5.5 (dfbe59da74e9c730a0af97c78a6e11dd336153ac)

limited shared resource only capable of processing a few thousand MSAs per day. Please
submit jobs only from a single IP address. We reserve the right to limit access to the
server case-by-case when usage exceeds fair use. If you require more MSAs: You can 
precompute all MSAs with `colabfold_search` or host your own API and pass it to `--host-url`

COMPLETE: 100% 150/150 [00:01<00:00, 96.27it/s] 
2025-08-26 14:

In [36]:
!mkdir -p /content/alphafold3_examples/input
!jq -r '.name |= "O00244"' \
  /content/alphafold3_examples/colabfold_msas/sp_O00244_ATOX1_HUMAN_Copper_transport_protein_ATOX1_OS_Homo_sapiens_OX_9606_GN_ATOX1_PE_1_SV_1.json \
  > /content/alphafold3_examples/input/O00244.json
!jq -r '.name |= "P0CG48"' \
  /content/alphafold3_examples/colabfold_msas/sp_P0CG48_UBC_HUMAN_Polyubiquitin-C_OS_Homo_sapiens_OX_9606_GN_UBC_PE_1_SV_3.json \
  > /content/alphafold3_examples/input/P0CG48.json
!jq -r '.name |= "Q01973"' \
  /content/alphafold3_examples/colabfold_msas/sp_Q01973_ROR1_HUMAN_Inactive_tyrosine-protein_kinase_transmembrane_receptor_ROR1_OS_Homo_sapiens_OX_9606_GN_ROR1_PE_1_SV_2.json \
  > /content/alphafold3_examples/input/Q01973.json

In [37]:
%%shell
source /alphafold3_venv/bin/activate
cd /app/alphafold
XLA_PYTHON_CLIENT_PREALLOCATE=true
XLA_CLIENT_MEM_FRACTION=0.95
python3 run_alphafold.py \
  --input_dir=/content/alphafold3_examples/input/ \
  --output_dir=/content/alphafold3_examples/output \
  --norun_data_pipeline

I0826 14:14:13.309282 20999316038272 xla_bridge.py:895] Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
I0826 14:14:13.310289 20999316038272 xla_bridge.py:895] Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory

Running AlphaFold 3. Please note that standard AlphaFold 3 model parameters are
only available under terms of use provided at
https://github.com/google-deepmind/alphafold3/blob/main/WEIGHTS_TERMS_OF_USE.md.
If you do not agree to these terms and are using AlphaFold 3 derived model
parameters, cancel execution of AlphaFold 3 inference with CTRL-C, and do not
use the model parameters.

Found local devices: [CudaDevice(id=0)], using device 0: cuda:0
Building model from scratch...
Checking that model parameters can be loaded...
2025-08-26 14:14:13.345900: W external/xla/xla/service/gpu/nvptx_compiler.cc:930] The NVIDIA driver's CUDA vers